In [ ]:
%pip install fastbook transformers tokenizers==0.12.0 torch

In [ ]:

import fastbook
import torch
from fastai.vision.all import *
from fastbook import *
import numpy as np
import numpy.random as nprnd

fastbook.setup_book()
model_name = "microsoft/deberta-v3-small"

In [ ]:
from datasets import load_dataset,Dataset
from transformers import AutoTokenizer
import pandas as pd
tokenizer = AutoTokenizer.from_pretrained(model_name)
from datasets import load_dataset

def concat_columns(example):
    example['input'] = 'TEXT1: ' + example["context"] + '; TEXT2: ' + example["target"] + '; ANC1: ' + example["anchor"]
    return example

def process_dataset(file):
    df = pd.read_csv(file)
    df["input"] = 'TEXT1: ' + df["context"] + '; TEXT2: ' + df["target"] + '; ANC1: ' + df["anchor"]
    train_dataset = Dataset.from_pandas(df)
    return train_dataset
test_dataset = process_dataset('./test.csv')
train_dataset = process_dataset('./train.csv')


In [ ]:
def encode(examples):
    return tokenizer(examples['input'])

train_dataset = train_dataset.map(encode, batched=True)
test_dataset = test_dataset.map(encode, batched=True)
print(test_dataset.column_names)

In [ ]:
train_dataset = train_dataset.rename_columns({'score':'labels'})


In [ ]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
lr = 8e-5
bs = 128
epochs = 4
# Define the training arguments
training_args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

# Create the Trainer and train the model
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,          # evaluation dataset,
    tokenizer=tokenizer, 
    compute_metrics=corr_d
)

trainer.train()



In [ ]:
np.set_printoptions(precision=2)
preds = trainer.predict(test_dataset).predictions.astype(float)
preds

In [ ]:
preds = np.clip(preds, 0, 1)
preds